In [9]:
import numpy as np, random, json, pickle, datetime, copy, socket, os, sys
from scipy.stats import sem
import matplotlib.colors as colors
from importlib import reload
import matplotlib.pyplot as plt

from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

if socket.gethostname() == 'Tolman':
    codeDirBase = 'C:\\Users\\whockei1\\Google Drive'
elif socket.gethostname() == 'DESKTOP-BECTOJ9':
    codeDirBase = 'C:\\Users\\whock\\Google Drive'
    
sys.path.insert(0, codeDirBase + '\\KnierimLab\\Ratterdam\\Code')
sys.path.insert(0, codeDirBase + '\\Python_Code\\KLab\\mts_analysis')
import utility_fx as util
import ratterdam_ParseBehavior as pBehav
import ratterdam_CoreDataStructures as core
import ratterdam_PermutationTests as Perm
import ratterdam_Directionality as Dir
import ratterdam_visBasic as Vis
from ratterdam_Defaults import *

cmap = util.makeCustomColormap()
plot = Vis.BasicRateMaps()

%qtconsole --style native

In [2]:
exp = "RFD5"
dayCode = f"R765{exp}\\"
figpath = f"C:\\Users\\whockei1\\Google Drive\\KnierimLab\\Ratterdam\\Figures\\R765{exp}\\"
datafile = f'E:\\Ratterdam\\R765\\R765{exp}\\'
behav = core.BehavioralData(datafile, exp, velocity_filter_thresh)
ts, position, alleyTracking, alleyVisits,  txtVisits = behav.loadData()

In [42]:
unit = core.UnitData("TT4\\cl-maze1.2", datafile, exp, alleyBounds, alleyVisits, txtVisits, position, ts)
unit.loadData_raw()

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:139: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:139: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:309: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:305: RuntimeWarning: invalid value encountered

In [3]:
intersectionBounds

{'A': [[105, 385], [165, 345], [165, 385], [140, 340], [105, 340]],
 'B': [[210, 385], [295, 390], [295, 352], [270, 340], [230, 340], [165, 345]],
 'C': [[359, 390], [430, 390], [430, 347], [410, 327], [370, 327], [359, 352]],
 'D': [[492, 390], [540, 390], [540, 325], [500, 325], [492, 347]],
 'E': [[105, 272], [140, 272], [150, 250], [150, 210], [145, 185], [105, 185]],
 'F': [[230, 275],
  [270, 275],
  [280, 250],
  [280, 210],
  [275, 205],
  [230, 205],
  [215, 210],
  [215, 250]],
 'G': [[370, 260],
  [410, 260],
  [420, 254],
  [420, 210],
  [410, 205],
  [370, 205],
  [350, 210],
  [350, 250]],
 'H': [[500, 260], [540, 260], [550, 205], [500, 205], [480, 210], [480, 254]],
 'I': [[105, 115], [145, 115], [165, 115], [165, 75], [105, 75]],
 'J': [[230, 132], [275, 132], [280, 115], [280, 75], [230, 75], [230, 115]],
 'K': [[370, 135], [410, 135], [415, 115], [415, 75], [350, 75], [350, 115]],
 'L': [[500, 140], [550, 140], [550, 75], [480, 75], [480, 115]]}

In [17]:
fig, ax = plt.subplots()

patches = []
for vertex in intersectionBounds.values():
    print(np.asarray(vertex).shape)
    poly = Polygon(np.asarray(vertex), True)
    patches.append(poly)
    ax.add_patch(poly)

(5, 2)
(6, 2)
(6, 2)
(5, 2)
(6, 2)
(8, 2)
(8, 2)
(6, 2)
(5, 2)
(6, 2)
(6, 2)
(5, 2)


In [128]:
plt.figure()
for coord in intersectionBounds.values():
    coord.append(coord[0]) #repeat the first point to create a 'closed loop'
    xs, ys = zip(*coord) #create lists of x and y values
    plt.plot(xs,ys)

In [44]:
intersectionList = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
intersectionTracking = {i:np.empty((0,3)) for i in intersectionList}
for intersection in intersectionList:
    p = path.Path(intersectionBounds[intersection])
    inOrNot = p.contains_points(position[:,1:])
    intersectionTracking[intersection] = position[inOrNot]

In [244]:
allAlleyVisits = []
for alley,visitList in alleyVisits.items():
    visits = [(alley+1,i, v[0], v[1]) for i,v in enumerate(visitList)]
    allAlleyVisits.extend(visits)
allAlleyVisits = sorted(allAlleyVisits, key = lambda x: x[2])

In [4]:
def getIntersectionPos(array,start,end,position):
    tsVisit = util.getTsinInverval(array, start, end)
    posVisit = util.getPosFromTs(tsVisit, position)
    return np.column_stack((tsVisit, posVisit))

In [5]:
def getIntersectionVisit(visit, visitPrev):
    """
    Returns spike array (ts,x,y) and occ array (ts,x,y)
    for the visit to an intersection.
    
    The arguments visit and visitPrev refer to
    the alley visits that "bookend" the intersection visit
    """
    start_int, end_int = visitprev[3], visit[2] #i.e. intersection visit starts at end of previous alley visit and goes to start of next alley visit
    intSpikes = getIntersectionPos(unit.spikes[:,0], start_int, end_int, position)
    intOccs = getIntersectionPos(position[:,0], start_int, end_int, position)
    return intSpikes, intOccs

In [131]:
visit, visitprev = allAlleyVisits[1], allAlleyVisits[0]
s,o = getIntersectionVisit(visit, visitprev)

In [98]:
ho=np.histogram2d(o[:,2],o[:,1],bins=30)[0]
hs = np.histogram2d(s[:,2],s[:,1],bins=30)[0]
n = (hs/ho)*33
plt.imshow(n)

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
def makeIntersectionRatemap(s,o,alley,dim=1):
    ho=np.histogram2d(o[:,2],o[:,1],bins=30)[0]
    hs = np.histogram2d(s[:,2],s[:,1],bins=30)[0]
    if unit.alleys[alley][0]['ratemap2d'].shape[0] > unit.alleys[alley][0]['ratemap2d'].shape[1]:
        a=1
    else:
        a=0
    ls,lo  = np.sum(hs,axis=a), np.sum(ho,axis=a)
    n = (ls* np.reciprocal(lo)) * 33  
    n = util.weird_smooth(n,smoothing_1d_sigma)
    n[np.where(lo==0)] = np.nan
    return n

In [43]:
intersectionRMs = {i:{txt:{"NE":np.empty((0,30)), "SW":np.empty((0,30))} for txt in ['A','B','C']} for i in range(1,18)}
for i in range(1,len(allAlleyVisits)):
    visit = allAlleyVisits[i]
    visitprev = allAlleyVisits[i-1]
    alley = visit[0]
    s,o = getIntersectionVisit(visit, visitprev)
    rm = makeIntersectionRatemap(s,o,alley)
    bounds = Dir.extractCorners(alleyBounds[alley-1])
    side = Dir.checkVisitEntrySide(unit.alleys[alley][visit[1]]['occs'], bounds)
    txt = unit.alleys[alley][visit[1]]['metadata']['stimulus'][0]
    intersectionRMs[alley][txt][side] = np.vstack((intersectionRMs[alley][txt][side], rm))

C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in reciprocal
  if __name__ == '__main__':
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in multiply
  if __name__ == '__main__':
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:305: RuntimeWarning: invalid value encountered in multiply
  W=0*U.copy()+1
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:309: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


In [17]:
fig, ax = plt.subplots(5,4,figsize=(7,6))

for alley in range(1,18):
    
    axis = fig.axes[alley-1]
    
    for txt,color in zip(['A','B','C'], ['r','b','g']):
        
        alleyrm, alleyerr = plot.compute_AvgTrace(unit, alley, txt)
        
        linNE, linSW = intersectionRMs[alley][txt]['NE'], intersectionRMs[alley][txt]['SW']
        linNEmean, linSWmean = np.nanmean(linNE,axis=0),  np.nanmean(linSW,axis=0)
        errNE, errSW = sem(linNE, axis=0, nan_policy='omit'), sem(linSW, axis=0, nan_policy='omit')
        expandedRM = np.concatenate((linSWmean, alleyrm, linNEmean))
        expandederr = np.concatenate((errSW, alleyerr, linNEmean))
        
        axis.plot(expandedRM, color=color)
        axis.fill_between(range(len(expandedRM)), expandedRM+expandederr, expandedRM-expandederr, alpha=0.5, color=color)
        
            

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_visBasic.py:89: RuntimeWarning: Mean of empty slice
  avg = np.nanmean(lins, axis=0)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:125: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: R

In [49]:
fig, ax = plt.subplots()
for alley in [10]:
    
    axis = ax
    
    for txt,color in zip(['A','B','C'], ['r','b','g']):
        
        alleyrm, alleyerr = plot.compute_AvgTrace(unit, alley, txt)
        
        linNE, linSW = intersectionRMs[alley][txt]['NE'], intersectionRMs[alley][txt]['SW']
        linNEmean, linSWmean = np.nanmean(linNE,axis=0),  np.nanmean(linSW,axis=0)
        errNE, errSW = sem(linNE, axis=0, nan_policy='omit'), sem(linSW, axis=0, nan_policy='omit')
        
        #expandedRM = np.concatenate((linSWmean, alleyrm, linNEmean))
        #expandederr = np.concatenate((errSW, alleyerr, linNEmean))
        
        expandedRM = np.concatenate((np.flip(linNEmean,axis=0), alleyrm, np.flip(linSWmean,axis=0)))
        expandederr = np.concatenate((np.flip(errNE,axis=0), alleyerr, np.flip(linSWmean,axis=0)))
        
        axis.plot(expandedRM, color=color,linewidth=0.5)
        axis.fill_between(range(len(expandedRM)), expandedRM+expandederr, expandedRM-expandederr, alpha=0.4, color=color)
        
    axis.vlines(30, 0, axis.get_ylim()[1],linestyle='--')
    axis.vlines(60, 0, axis.get_ylim()[1],linestyle='--')
        
            

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:309: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:305: RuntimeWarning: invalid value encountered in multiply
  W=0*U.copy()+1
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()


In [253]:
for txt,c in zip(['A','B','C'], ['r','b','g']):
    for v in intersectionRMs[8][txt]['SW']:
        plt.plot(v,color=c)

In [69]:
exp = "RFD3"
dayCode = f"R765{exp}"
datafile = f'E:\\Ratterdam\\R765\\R765{exp}\\'
behav = core.BehavioralData(datafile, exp, velocity_filter_thresh)
ts, position, alleyTracking, alleyVisits,  txtVisits = behav.loadData()

savepath = f"C:\\Users\\whockei1\\Google Drive\\KnierimLab\\Ratterdam\\Figures\\{dayCode}\\velocityFiltered_{velocity_filter_thresh}\\_withIntersections\\"
if not os.path.isdir(savepath):
    os.makedirs(savepath)
    
unitList = util.getClustList(datafile)
plot = Vis.BasicRateMaps()

allAlleyVisits = []
for alley,visitList in alleyVisits.items():
    visits = [(alley+1,i, v[0], v[1]) for i,v in enumerate(visitList)]
    allAlleyVisits.extend(visits)
allAlleyVisits = sorted(allAlleyVisits, key = lambda x: x[2])

for unitName in unitList:
     
    unit = core.UnitData(unitName, datafile, exp, alleyBounds, alleyVisits, txtVisits, position, ts)
    unit.loadData_raw()     
        

    intersectionRMs = {i:{txt:{"NE":np.empty((0,30)), "SW":np.empty((0,30))} for txt in ['A','B','C']} for i in range(1,18)}
    for i in range(1,len(allAlleyVisits)):
        visit = allAlleyVisits[i]
        visitprev = allAlleyVisits[i-1]
        alley = visit[0]
        s,o = getIntersectionVisit(visit, visitprev)
        rm = makeIntersectionRatemap(s,o,alley)
        bounds = Dir.extractCorners(alleyBounds[alley-1])
        side = Dir.checkVisitEntrySide(unit.alleys[alley][visit[1]]['occs'], bounds)
        txt = unit.alleys[alley][visit[1]]['metadata']['stimulus'][0]
        intersectionRMs[alley][txt][side] = np.vstack((intersectionRMs[alley][txt][side], rm))
    
    
    fig, ax = plt.subplots(5,4,figsize=(12,11))

    for alley in range(1,18):

        axis = fig.axes[alley-1]

        for txt,color in zip(['A','B','C'], ['r','b','g']):

            alleyrm, alleyerr = plot.compute_AvgTrace(unit, alley, txt)

            linNE, linSW = intersectionRMs[alley][txt]['NE'], intersectionRMs[alley][txt]['SW']
            linNEmean, linSWmean = np.nanmean(linNE,axis=0),  np.nanmean(linSW,axis=0)
            errNE, errSW = sem(linNE, axis=0, nan_policy='omit'), sem(linSW, axis=0, nan_policy='omit')
            
            
            expandedRM = np.concatenate((np.flip(linNEmean,axis=0), alleyrm, np.flip(linSWmean,axis=0)))
            expandederr = np.concatenate((np.flip(errNE,axis=0), alleyerr, np.flip(linSWmean,axis=0)))

            axis.plot(expandedRM, color=color, linewidth=0.5)
            axis.fill_between(range(len(expandedRM)), expandedRM+expandederr, expandedRM-expandederr, alpha=0.4, color=color)
            axis.set_title(f"{alley}")
            
        axis.vlines(30, 0, axis.get_ylim()[1], alpha=0.6, linestyle='--')
        axis.vlines(60, 0, axis.get_ylim()[1], alpha=0.6, linestyle='--')
                                         
            
    try:
        plt.savefig(savepath+f"{unit.name}_alley_intersection.svg")
    except:
        pass
    plt.close()

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:139: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:139: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:309: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whockei1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by 

## Trial-by-trial directionality by texture analysis for a target alley that seems interesting

In [25]:
exp = "RFD3"
dayCode = f"R765{exp}"
datafile = f'E:\\Ratterdam\\R765\\R765{exp}\\'
behav = core.BehavioralData(datafile, exp, 0)
ts, position, alleyTracking, alleyVisits,  txtVisits = behav.loadData()

In [28]:
unit = core.UnitData("TT4\\cl-maze1.7", datafile, exp, alleyBounds, alleyVisits, txtVisits, position, ts)
unit.loadData_raw()

allAlleyVisits = []
for alley,visitList in alleyVisits.items():
    visits = [(alley+1,i, v[0], v[1]) for i,v in enumerate(visitList)]
    allAlleyVisits.extend(visits)
allAlleyVisits = sorted(allAlleyVisits, key = lambda x: x[2])


intersectionRMs = {i:{txt:{"NE":np.empty((0,30)), "SW":np.empty((0,30))} for txt in ['A','B','C']} for i in range(1,18)}
alleyVisitNums = {i:{txt:{"NE":[], "SW":[]} for txt in ['A','B','C']} for i in range(1,18)}
for i in range(1,len(allAlleyVisits)):
    visit = allAlleyVisits[i]
    visitprev = allAlleyVisits[i-1]
    alley = visit[0]
    s,o = getIntersectionVisit(visit, visitprev)
    rm = makeIntersectionRatemap(s,o,alley)
    bounds = Dir.extractCorners(alleyBounds[alley-1])
    side = Dir.checkVisitEntrySide(unit.alleys[alley][visit[1]]['occs'], bounds)
    txt = unit.alleys[alley][visit[1]]['metadata']['stimulus'][0]
    intersectionRMs[alley][txt][side] = np.vstack((intersectionRMs[alley][txt][side], rm))
    alleyVisitNums[alley][txt][side].append(visit[1])

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:139: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:139: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:145: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:309: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whockei1\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:305: RuntimeWarning: invalid value encountered

In [29]:
target_alley = 11

fig, ax = plt.subplots(3,2, figsize=(5,5))
i=0
allvs = np.empty((0,30))
for txt in ['A', 'B', 'C']:
    for side in ["SW", "NE"]:
        if intersectionRMs[target_alley][txt][side].shape[0] > 0:
            for visit in intersectionRMs[target_alley][txt][side]:
                allvs = np.vstack((allvs, visit))
mymax = util.calcSmartMax(allvs, cutoff=0.98, scale=2.5, bins=100)

for txt in ['A', 'B', 'C']:
    for side in ["SW", "NE"]:
        if intersectionRMs[target_alley][txt][side].shape[0] > 0:
            axis = fig.axes[i]
            im = axis.imshow(intersectionRMs[target_alley][txt][side], interpolation='None', aspect='auto', cmap=cmap,vmax=mymax)
            
            axis.set_yticks(range(len(alleyVisitNums[target_alley][txt][side])))
            axis.set_yticklabels(alleyVisitNums[target_alley][txt][side])
        i+=1
        
fig.axes[0].set_title("SW Entry")
fig.axes[1].set_title("NE Entry")
fig.axes[0].set_ylabel("A")
fig.axes[2].set_ylabel("B")
fig.axes[4].set_ylabel("C")

plt.suptitle(f"{exp} {unit.name} Alley {target_alley}")
plot.add_colorbar(fig,im)
plt.show()              

In [30]:
%qtconsole

In [ ]:
def checkCompletePass():
    pass

def checkAlleyOrientation():
    pass


